In [ ]:
import numpy as np

In [ ]:
%run /content/drive/MyDrive/Colab\ Notebooks/Konan_6_AdamModel.ipynb
%run /content/drive/MyDrive/Colab\ Notebooks/mathutil.ipynb

In [ ]:
class CnnBasicModel(AdamModel):
  def __init__(self, name, dataset, hconfigs, show_maps = False):
    if isinstance(hconfigs, list) and \
    not isinstance(hconfigs[0], (list, int)):
      hconfigs = [hconfigs]
    self.show_maps = show_maps
    self.need_maps = False
    self.kernels = []
    super(CnnBasicModel, self).__init__(name, dataset, hconfigs) # call self.init_parameters(hconfigs)
    self.use_adam = True

>Sub functions1  
>>get_layer_type  
>>get_conf_param  
>>get_conf_param_2d  
----
>Sub functions2  
>>cnn_basic_activate  
>>cnn_basic_activate_derv

In [ ]:
# SUB FUNCTIONS1
def get_layer_type(hconfig):
  if not isinstance(hconfig, list):
    return 'full'
  return hconfig[0]

def get_conf_param(hconfig, key, defval=None):
  if not isinstance(hconfig, list):
    return defval
  if len(hconfig) <= 1:
    return defval
  if not key in hconfig[1]:
    return defval
  return hconfig[1][key]

def get_conf_param_2d(hconfig, key, defval=None):
  if len(hconfig) <= 1:
    return defval
  if not key in hconfig[1]:
    return defval
  val = hconfig[1][key]
  if isinstance(val, list):
    return val
  return [val, val]

# SUB FUNCTIONS2
def cnn_basic_activate(self, affine, hconfig):
  if hconfig is None:
    return affine
  func = get_conf_param(hconfig, 'actfunc', 'relu')
  if func == 'none':
    return affine
  elif func == 'relu':
    return relu(affine)
  elif func == 'sigmoid':
    return sigmoid(affine)
  elif func == 'tanh':
    return tanh(affine)
  assert 0

def cnn_basic_activate_derv(self, G_y, y, hconfig):
  if hconfig is None:
    return G_y
  func = get_conf_param(hconfig, 'actfunc', 'relu')
  if func == 'none':
    return G_y
  elif func == 'relu':
    return relu_derv(y) * G_y
  elif func == 'sigmoid':
    return sigmoid_derv(y) * G_y
  elif func == 'tanh':
    return tanh_derv(y) * G_y
  assert 0

CnnBasicModel.activate = cnn_basic_activate
CnnBasicModel.activate_derv = cnn_basic_activate_derv

>cnn_basic_alloc_layer_param  
>>cnn_basic_alloc_full_layer  
>>cnn_basic_alloc_conv_layer  
>>cnn_basic_alloc_pool_layer  

In [ ]:
def cnn_basic_alloc_layer_param(self, input_shape, hconfig):
  layer_type = get_layer_type(hconfig)
  m_name = 'alloc_{}_layer'.format(layer_type)
  method = getattr(self, m_name)
  pm, output_shape = method(input_shape, hconfig)
  return pm, output_shape

CnnBasicModel.alloc_layer_param = cnn_basic_alloc_layer_param

In [ ]:
def cnn_basic_alloc_full_layer(self, input_shape, hconfig):
  input_cnt = np.prod(input_shape)
  output_cnt = get_conf_param(hconfig, 'width', hconfig)
  weight = np.random.normal(0, self.rand_std, [input_cnt, output_cnt])
  bias = np.zeros([output_cnt])
  return {'w':weight, 'b':bias}, [output_cnt]

def cnn_basic_alloc_conv_layer(self, input_shape, hconfig):
  assert len(input_shape) == 3
  xh, xw, xchn = input_shape
  kh, kw = get_conf_param_2d(hconfig, 'ksize')
  ychn = get_conf_param(hconfig, 'chn')

  kernel = np.random.normal(0, self.rand_std, [kh, kw, xchn, ychn])
  bias = np.zeros([ychn])
  if self.show_maps:
    self.kernels.append(kernel)
  return {'k':kernel, 'b':bias}, [xh, xw, ychn]

def cnn_basic_alloc_pool_layer(self, input_shape, hconfig):
  assert len(input_shape) == 3
  xh, xw, xchn = input_shape
  sh, sw = get_conf_param_2d(hconfig, 'stride')

  assert xh % sh == 0
  assert xw % sw == 0

  return {}, [xh//sh, xw//sw, xchn]

CnnBasicModel.alloc_full_layer = cnn_basic_alloc_full_layer
CnnBasicModel.alloc_conv_layer = cnn_basic_alloc_conv_layer
CnnBasicModel.alloc_max_layer = cnn_basic_alloc_pool_layer
CnnBasicModel.alloc_avg_layer = cnn_basic_alloc_pool_layer

>cnn_basic_forward_layer
>>cnn_basic_forward_full_layer  
>>cnn_basic_forward_cnn_layer  
>>cnn_basic_forward_avg_layer  
>>cnn_basic_forward_max_layer  
----
>cnn_basic_backprop_layer  
>>cnn_basic_backprop_full_layer  
>>cnn_basic_backprop_cnn_layer  
>>cnn_basic_backprop_avg_layer  
>>cnn_basic_backprop_max_layer  

In [ ]:
def cnn_basic_forward_layer(self, x, hconfig, pm):
  layer_type = get_layer_type(hconfig)
  m_name = 'forward_{}_layer'.format(layer_type)
  method = getattr(self, m_name)
  y, aux = method(x, hconfig, pm)
  return y, aux

CnnBasicModel.forward_layer = cnn_basic_forward_layer

In [ ]:
def cnn_basic_backprop_layer(self, G_y, hconfig, pm, aux):
  layer_type = get_layer_type(hconfig)
  m_name = 'backprop_{}_layer'.format(layer_type)
  method = getattr(self, m_name)
  G_input = method(G_y, hconfig, pm, aux)
  return G_input
CnnBasicModel.backprop_layer = cnn_basic_backprop_layer

In [ ]:
def cnn_basic_forward_full_layer(self, x, hconfig, pm):
  if pm is None:      # for GAN
    return x, hconfig
  x_org_shape = x.shape
  if len(x.shape) != 2:   # In case of turning from CNN to Affine
    mb_size = x.shape[0]
    x = x.reshape([mb_size, -1])
  affine = np.matmul(x, pm['w']) + pm['b']
  y = self.activate(affine, hconfig)

  return y, [x, y, x_org_shape]

def cnn_basic_backprop_full_layer(self, G_y, hconfig, pm, aux):
  if pm is None:      # for GAN
    return G_y  
  x, y, x_org_shape = aux
  G_affine = self.activate_derv(G_y, y, hconfig)  # G_affine.shape : N, output

  G_weight = np.matmul(x.T, G_affine)
  G_bias = np.sum(G_affine, axis=0)
  G_input = np.matmul(G_affine, pm['w'].T)

  self.update_param(pm, 'w', G_weight)
  self.update_param(pm, 'b', G_bias)
  return G_input.reshape(x_org_shape)

CnnBasicModel.forward_full_layer = cnn_basic_forward_full_layer
CnnBasicModel.backprop_full_layer = cnn_basic_backprop_full_layer

In [ ]:
def get_ext_regions_for_conv(x, kh, kw):
  mb_size, xh, xw, xchn = x.shape
  regs = get_ext_regions(x, kh, kw, 0)      #regs.shape : xh, xw, mb_size, kh, kw, xchn
  regs = regs.transpose([2, 0, 1, 3, 4, 5]) #regs.shape : mb_size, xh, xw, kh, kw, xchn
  return regs.reshape(mb_size * xh * xw, kh*kw*xchn)  #regs.shape : mb_size x xh x xw, kh x kw x xchn

def get_ext_regions(x, kh, kw, fill):
  mb_size, xh, xw, xchn = x.shape

  eh, ew = xh + kh - 1, xw + kw - 1
  bh, bw = (kh-1)//2, (kw-1)//2

  x_ext = np.zeros((mb_size, eh, ew, xchn), dtype='float32') + fill
  x_ext[:, bh:bh+xh, bw:bw+xw, :] = x
  
  regs = np.zeros((xh, xw, mb_size * kh * kw * xchn), dtype="float32")

  for i in range(xh):
    for j in range(xw):
      regs[i, j, :] = x_ext[:, i:i+kh, j:j+kw, :].flatten()
  return regs.reshape(xh, xw, mb_size, kh, kw, xchn)

In [ ]:
def forward_conv_layer_adhoc(self, x, hconfig, pm):
  mb_size, xh, xw, xchn = x.shape
  kh, kw, _, ychn = pm['k'].shape

  conv = np.zeros((mb_size, xh, xw, ychn))
  for n in range(mb_size):
    for x_i in range(xh):
      for x_j in range(xw):
        for ychn_i in range(ychn):
          for k_i in range(kh):
            for k_j in range(kw):
              i = x_i + k_i - (kh - 1) // 2
              j = x_j + k_j - (kw - 1) // 2
              if i < 0 or i >= xh:
                continue
              if j < 0 or j >= xw:
                continue
              for xchn_i in range(xchn):
                kval = pm['k'][k_i][k_j][xchn_i][ychn_i]
                xval = x[n][x_i][x_j][xchn_i]
                conv[n][x_i][x_j][ychn_i] += kval * xval
  y = self.activate(conv + pm['b'], hconfig)
  return y, [x, y]

def forward_conv_layer_better(self, x, hconfig, pm):
  mb_size, xh, xw, xchn = x.shape
  kh, kw, _, ychn = pm['k'].shape

  conv = np.zeros((mb_size, xh, xw, ychn))

  bh, bw = (kh-1)//2, (kw-1)//2
  eh, ew = xh + kh - 1, xw + kw - 1

  x_ext = np.zeros((mb_size, eh, ew, xchn))
  x_ext[:, bh:bh+xh, bw:bw+xw, :] = x
  k_flat = pm['k'].transpose([3, 0, 1, 2]).reshape([ychn, -1])

  for n in range(mb_size):
    for xh_ in range(xh):
      for xw_ in range(xw):
        for ychn_ in range(ychn):
          xe_flat = x_ext[n, xh_:xh_+kh, xw_:xw_+kw, :].flatten()
          # xe_flat.shape : mb_size, xh + kh - 1, xw + kw - 1, xchn
          conv[n, xh_, xw_, ychn_] = (xe_flat * k_flat[ychn_]).sum()
  y = self.activate(conv + pm['b'], hconfig)
  return y, [x, y]

def cnn_basic_forward_conv_layer(self, x, hconfig, pm):
  mb_size, xh, xw, xchn = x.shape
  kh, kw, _, ychn = pm['k'].shape

  x_flat = get_ext_regions_for_conv(x, kh, kw)  # x_flat.shape : mb_size x xh x xw, kh x kw x xchn
  k_flat = pm['k'].reshape([kh*kw*xchn, ychn])  # k_flat.shape : kh x kw x xchn, ychn
  conv_flat = np.matmul(x_flat, k_flat)         # conv_flat : mb_size x xh x xw, ychn
  conv = conv_flat.reshape([mb_size, xh, xw, ychn])
  y = self.activate(conv + pm['b'], hconfig)
  if self.need_maps:
    self.maps.append(y)
  return y, [x_flat, k_flat, x, y]

CnnBasicModel.forward_conv_layer = cnn_basic_forward_conv_layer

In [ ]:
def undo_ext_regions_for_conv(regs, x, kh, kw):
  mb_size, xh, xw, xchn = x.shape
  regs = regs.reshape(mb_size, xh, xw, kh, kw, xchn)
  regs = regs.transpose([1,2,0,3,4,5])  #regs.shape : xh, xw, mb_size, kh, kw, xchn
  return undo_ext_regions(regs, kh, kw)

def undo_ext_regions(regs, kh, kw):
  xh, xw, mb_size, kh, kw, xchn = regs.shape
  eh, ew = xh + kh - 1, xw + kw -1
  bh, bw = (kh-1)//2, (kw-1)//2

  gx_ext = np.zeros([mb_size, eh, ew, xchn], dtype="float32")

  for i in range(xh):
    for j in range(xw):
      gx_ext[:, i:i+kh, j:j+kw, :] += regs[i, j]
  return gx_ext[:, bh:bh+xh, bw:bw+xw, :]

In [ ]:
def cnn_basic_backprop_conv_layer(self, G_y, hconfig, pm, aux):
  x_flat, k_flat, x, y = aux
  kh, kw, xchn, ychn = pm['k'].shape
  mb_size, xh, xw, _ = G_y.shape
  # x_flat.shape : mb_size x xh x xw, kh x kw x xchn
  # k_flat.shape : kh x kw x xchn, ychn 
  G_conv = self.activate_derv(G_y, y, hconfig)    # G_conv.shape : mb_size, xh, xw, ychn
  G_conv_flat = G_conv.reshape(mb_size*xh*xw, ychn)  # G_conv_flat.shape : mb_size x xh x xw, ychn

  G_k_flat = np.matmul(x_flat.T, G_conv_flat)     # G_k_flat.shape : kh x kw x xchn, ychn
  G_x_flat = np.matmul(G_conv_flat, k_flat.T)     # G_x_flat.shape : mb_size x xh x xw, kh x kw x xchn
  G_bias = np.sum(G_conv_flat, axis=0)            # G_bias.shape : ychn

  G_kernel = G_k_flat.reshape(kh, kw, xchn, ychn)
  G_input = undo_ext_regions_for_conv(G_x_flat, x, kh, kw)

  self.update_param(pm, 'k', G_kernel)
  self.update_param(pm, 'b', G_bias)

  return G_input

CnnBasicModel.backprop_conv_layer = cnn_basic_backprop_conv_layer

In [ ]:
def cnn_basic_forward_avg_layer(self, x, hconfig, pm):
  mb_size, xh, xw, chn = x.shape
  sh, sw = get_conf_param_2d(hconfig, 'stride')
  yh, yw = xh // sh, xw // sw

  x1 = x.reshape(mb_size, yh, sh, yw, sw, chn)
  x2 = x1.transpose(0,1,3,5,2,4)
  x3 = x2.reshape(-1, sh * sw)

  y_flat = np.average(x3, axis=1)
  y = y_flat.reshape(mb_size, yh, yw, chn)
  if self.need_maps:
    self.maps.append(y)
  return y, None  # return output, aux

def cnn_basic_backprop_avg_layer(self, G_y, hconfig, pm, aux):
  mb_size, yh, yw, chn = G_y.shape
  sh, sw = get_conf_param_2d(hconfig, 'stride')
  xh, xw = yh * sh, yw * sw

  gy_flat = G_y.flatten() / (sh * sw) # gy_flat.shape : mb_size x yh x yw x chn
  gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype="float32")
  for i in range(sh * sw):
    gx1[:, i] = gy_flat
  gx2 = gx1.reshape(mb_size, yh, yw, chn, sh, sw)
  gx3 = gx2.transpose(0, 1, 4, 2, 5, 3)
  G_input = gx3.reshape(mb_size, xh, xw, chn)
  return G_input

CnnBasicModel.forward_avg_layer = cnn_basic_forward_avg_layer
CnnBasicModel.backprop_avg_layer = cnn_basic_backprop_avg_layer

In [ ]:
def cnn_basic_forward_max_layer(self, x, hconfig, pm):
  mb_size, xh, xw, chn = x.shape
  sh, sw = get_conf_param_2d(hconfig, 'stride')
  yh, yw = xh // sh, xw // sw

  x1 = x.reshape(mb_size, yh, sh, yw, sw, chn)
  x2 = x1.transpose(0,1,3,5,2,4)
  x3 = x2.reshape(-1, sh * sw)

  idxs = np.argmax(x3, axis=1)
  y_flat = x3[np.arange(mb_size*yh*yw*chn), idxs]
  y = y_flat.reshape(mb_size, yh, yw, chn)
  if self.need_maps:
    self.maps.append(y)
  return y, idxs

def cnn_basic_backprop_max_layer(self, G_y, hconfig, pm, aux):
  idxs = aux
  mb_size, yh, yw, chn = G_y.shape
  sh, sw = get_conf_param_2d(hconfig, 'stride')
  xh, xw = yh * sh, yw * sw

  gy_flat = G_y.flatten()
  gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype="float32")
  gx1[np.arange(mb_size*yh*yw*chn), idxs] = gy_flat[:]
  gx2 = gx1.reshape(mb_size, yh, yw, chn, sh, sw)
  gx3 = gx2.transpose(0, 1, 4, 2, 5, 3)

  G_input = gx3.reshape(mb_size, xh, xw, chn)
  return G_input

CnnBasicModel.forward_max_layer = cnn_basic_forward_max_layer
CnnBasicModel.backprop_max_layer = cnn_basic_backprop_max_layer

In [ ]:
def cnn_basic_visualize(self, num):
  print('Model {} visualization'.format(self.name))
  self.need_maps = self.show_maps
  self.maps = []
  deX, deY = self.dataset.get_visualize_data(num)
  est = self.get_estimate(deX)
  if self.show_maps:
    for kernel in self.kernels:
      kh, kw, xchn, ychn = kernel.shape
      grids = kernel.reshape(kh, kw, -1).transpose(2, 0, 1)  # grids.shape : xchn*ychn, kh, kw
      draw_images_horz(grids[0:5, :, :])
    for pmap in self.maps:
      draw_images_horz(pmap[:,:,:,0]) 
  self.dataset.visualize(deX, est, deY)
  self.need_maps = False
  self.maps = None

CnnBasicModel.visualize = cnn_basic_visualize

In [ ]:
def eval_stride_shape(hconfig, conv_type, xh, xw, ychn):
  kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)
  if padding == 'VALID':
    xh = xh - kh + 1
    xw = xw - kw + 1
  yh = xh // sh
  yw = xw // sw
  return [yh, yw, ychn]

In [ ]:
def stride_filter(hconfig, conv_type, y):
  _, xh, xw, _ = x_shape = y.shape
  nh, nw = xh, xw
  kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)

  if padding == 'VALID':
    bh, bw = (kh - 1)//2, (kw - 1)//2
    nh, nw = xh - kh + 1, xw - kw + 1
    y = y[:, bh:bh+nh, bw:bw+nw, :]
  
  if sh != 1 or sw != 1:
    bh, bw = (kh - 1)//2, (kw - 1)//2
    mh, mw = nh // sh, nw // sw
    y = y[:, bh:bh + sh*mh:sh, bw:bw + sw*mw:sw, :]
  return y, [x_shape, nh, nw]

In [ ]:
def stride_filter_derv(hconfig, conv_type, G_y, aux):
  x_shape, nh, nw = aux
  mb_size, xh, xw, xchn = x_shape
  kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)

  if sh != 1 or sw != 1:
    bh, bw = (kh - 1)//2, (kw - 1)//2
    mh, mw = nh // sh, nw // sw
    G_y_tmp = np.zeros((mb_size, nh, nw, xchn))
    G_y_tmp[:, bh:bh + sh*mh:sh, bw:bw + sw*mw:sw, :] = G_y
    G_y = G_y_tmp
  
  if padding == 'VALID':
    bh, bw = (kh - 1)//2, (kw - 1)//2
    nh, nw = xh - kh + 1, xw - kw + 1
    G_y_tmp = np.zeros((mb_size, xh, xw, xchn))
    G_y_tmp[:, bh:bh+nh, bw:bw+nw, :] = G_y
    G_y = G_y_tmp
  return G_y

In [ ]:
def get_shape_params(hconfig, conv_type):
  if conv_type:
    kh, kw = get_conf_param_2d(hconfig, 'ksize')
    sh, sw = get_conf_param_2d(hconfig, 'stride', [1, 1])
  else: # pooling_type
    sh, sw = get_conf_param_2d(hconfig, 'stride', [1, 1])
    kh, kw = get_conf_param_2d(hconfig, 'ksize', [sh, sw])
  padding = get_conf_param(hconfig, 'padding', 'SAME')

  return kh, kw, sh, sw, padding